# 파이썬으로 풀어보는 수학

## 7장 미적분 문제 풀기

### 1.  함수란 무엇인가 ?

`함수`란 입력집합과 출력집합 간의 관계(mapping)를 얘기합니다.
유요한 입력값의 집합을 함수 `도메인(Domain)`이라 하고, 결과집합을 `범위(Range)`라고 합니다.
예를 들어서 *f(x) = 1/x* 라는 함수가 있는 경우 x의 값은 0이 될 수 없습니다.
그러므로 이 함수의 도메인은 0이 아닌 모든 실수와 복소수 입니다.
결과집합인 범위도 0이 아닌 모든 실수와 복소수 입니다.
*f(x) = x^2*에 대해서는 도메인은 모든 양수와 음수이지만, 범위는 양수만 해당합니다.

### 2. 가정(assumptions)

`sympy`의 `Symbol`객체를 이용를 생성한 다음 해당 변수의 값을 가정하고 판단하는 것이 가능합니다.
*x+5* 값이 0보다 큰지 여부를 확인하는 코드를 작성해 보겠습니다.

In [1]:
from sympy import Symbol

x = Symbol('x')
if x + 5 > 0:
    print('Positive')
else:
    print('Negative')

TypeError: cannot determine truth value of Relational

`TypeError: cannot determine truth value of Relational`라는 오류가 발생합니다.
왜냐면 x의 값이 범위를 알 수 없으므로 해당 식이 0보다 큰지 작은지 추론을 할 수가 없기 때문입니다.
`Symbol` 객체를 `positive=True`로 설정하면 해당 식은 항상 0보다 클 것입니다.

In [2]:
x = Symbol('x', positive=True)
if x + 5 > 0:
    print('Positive')
else:
    print('Negative')

Positive


`negative=True`로 선택하면 참, 거짓이 모두 가능하기 때문에 처음과 같은 `TypeError: cannot determine truth value of Relational` 오류가 발생합니다.

In [3]:
x = Symbol('x', negative=True)
if x + 5 > 0:
    print('Positive')
else:
    print('Negative')

TypeError: cannot determine truth value of Relational

앞에서 살펴본 `positive`, `negative` 외에도 `real`, `integer`, `complex`, `imaginary` 등으로 설정이 가능합니다.

### 3. 함수의 극한

미적분 계산의 기본은 변수의 값이 어떤값에 근접할 때의 함수의 극한값을 찾는 것입니다.
`f(x) = 1/x` 함수의 경우 x가 증가할수록 0에 가까워 집니다.

![수식 추의주기](./image/DoingMathWithPython.Ch07.equation.01.png)

`sympy`에서는 극한을 `Limit` 객체를 이용해서 생성이 가능합니다.

In [5]:
from sympy import Limit, Symbol, S

x = Symbol('x')
L = Limit(1/x, x, S.Infinity)
L

Limit(1/x, x, oo, dir='-')

극한값 계산에는 `doit()`함수를 사용합니다.

In [6]:
L.doit()

0

0의 방향으로 극한값은 양의 방향에서 0으로 접근을하면 양의 무한대 값이 되며, 음의 방향에서 0으로 접근하면 양의 무한대 값이 됩니다.

In [7]:
Limit(1/x, x, 0, dir='-').doit()

-oo

In [8]:
Limit(1/x, x, 0, dir='+').doit()

oo

정해지지 않은 극한값도 다룰수 있습니다. ( *0/0 = 1*, *inf / inf = 0*)

In [9]:
from sympy import sin

Limit(sin(x)/x, x, 0).doit()

1

### 4. 연속 복리
